БИБЛИОТЕКИ

In [ ]:
import numpy as np
from tensorflow.keras.models import Model 
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import recall_score
from tensorflow.keras.callbacks import Callback
from sklearn.model_selection import train_test_split

ЧТЕНИЕ МАССИВОВ

In [ ]:
# монтируем свой диск...
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


ССЫЛКИ НА ФАЙЛЫ ДЛЯ ШАГА 3 - РЕГРЕССИЯ НА ОСТАТКЕ

In [ ]:
# входные данные
x_train_file = '/content/drive/My Drive/Bases/DTP/x_train_22.dat'
y_train_file = '/content/drive/My Drive/Bases/DTP/y_train_22.dat'

x_val_file = '/content/drive/My Drive/Bases/DTP/x_val_22.dat'
y_val_file = '/content/drive/My Drive/Bases/DTP/y_val_22.dat'


# выходный файл с весами лучшей модели
model_save_file = '/content/drive/My Drive/Bases/DTP/model_22.h5'

ЧТЕНИЕ ДАННЫХ

In [ ]:
# чтение
x_train = np.fromfile(x_train_file)
x_train = x_train.reshape((21935, 7))
y_train = np.fromfile(y_train_file)
y_train = y_train.reshape((21935, 4))

x_val = np.fromfile(x_val_file)
x_val = x_val.reshape((90, 7))
y_val = np.fromfile(y_val_file)
y_val = y_val.reshape((90, 4))

print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

(21935, 7)
(21935, 4)
(90, 7)
(90, 4)


МОДЕЛЬ НЕЙРОННОЙ СЕТИ

In [ ]:
def CreateModel(activation='linear', dropout=0.1):

  input = Input((7,)) 

  x = BatchNormalization()(input)
  x = Dense(200, activation=activation)(x)
  x = Dropout(dropout)(x)
  
  x = BatchNormalization()(x)
  x = Dense(200, activation=activation)(x)
  x = Dropout(dropout)(x)

  x = BatchNormalization()(x)
  x = Dense(200, activation=activation)(x)
  x = Dropout(dropout)(x)

  output_1 = Dense(1, activation='relu')(x) 
  output_2 = Dense(1, activation='relu')(x) 
  output_3 = Dense(1, activation='relu')(x) 
  output_4 = Dense(1, activation='relu')(x) 

  return Model(input, [output_1, output_2, output_3, output_4]) 

CALLBACK, сохраняющий лучший результат по метрике Recall

In [ ]:
class MyCallback(Callback):
  
  # инициализация
  def __init__(self, model, x_val, y_val):
    # модель
    self.model = model 
    # проверочная выборка
    self.x_val = x_val 
    self.y_val = y_val
    # начальный счет по метрике
    self.score = 0
       
  # подсчет метрики Recall для val  
  def calc_recall(self):
    y_pred = self.model.predict(self.x_val)
    y_p0 = np.around(y_pred[0]).astype(int)
    y_p1 = np.around(y_pred[1]).astype(int)
    y_p2 = np.around(y_pred[2]).astype(int)
    y_p3 = np.around(y_pred[3]).astype(int)
    y_t0 = np.around(self.y_val[:, 0]).astype(int)
    y_t1 = np.around(self.y_val[:, 1]).astype(int)
    y_t2 = np.around(self.y_val[:, 2]).astype(int)
    y_t3 = np.around(self.y_val[:, 3]).astype(int)
    score = 0
    score += recall_score(y_t0, y_p0, average='macro')
    score += recall_score(y_t1, y_p1, average='macro')
    score += recall_score(y_t2, y_p2, average='macro')
    score += recall_score(y_t3, y_p3, average='macro')
    return score / 4

  # окончение каждой эпохи....
  def on_epoch_end(self, epoch, logs=None):
    score_0 = self.calc_recall()
    print('\n\nТекущий Recall =', score_0, '\n')
    if score_0 > self.score:
      self.score = score_0
      print('\n\n!!!!!!!!Лучший Recall =', score_0, '\n')
      self.model.save_weights(model_save_file)   

  # в конце обучения
  def on_train_end(self, logs=None):
    print('\n\n!ЛУЧШИЙ Recall ОБУЧЕНИЯ =', self.score, '\n')


In [ ]:
# параметры обучения

batch_size = 410

dropout = 0.7

activate_function = 'relu'

epochs_count = 20

learning_rate = 1e-4

ОБУЧЕНИЕ МОДЕЛИ БИНАРНОЙ КЛАССИФИКАЦИИ ШАГ 1

In [ ]:
del model

In [ ]:
# создание модели
model = CreateModel(activation=activate_function, dropout=dropout)  
cb = MyCallback(model, x_val, y_val)
#cb = MyCallback(model, x_train, y_train)

In [ ]:
# компиляция
model.compile(optimizer=Adam(learning_rate=learning_rate), 
              loss=['mse', 'mse', 'mse', 'mse'], 
              metrics=['accuracy'])

# обучение
history = model.fit(x_train, [y_train[:, 0], y_train[:, 1], y_train[:, 2], y_train[:, 3]], 
                    batch_size = batch_size,
                    epochs= epochs_count, 
                    validation_data=(x_val, [y_val[:, 0], y_val[:, 1], y_val[:, 2], y_val[:, 3]]), 
                    verbose=1, 
                    callbacks=[cb])

Epoch 1/20
53/54 [============================>.] - ETA: 0s - loss: 7.8836 - dense_10_loss: 1.4275 - dense_11_loss: 0.1588 - dense_12_loss: 4.7140 - dense_13_loss: 1.5834 - dense_10_accuracy: 0.3991 - dense_11_accuracy: 0.8400 - dense_12_accuracy: 0.1173 - dense_13_accuracy: 0.4289

Текущий Recall = 0.5031193372781423 



!!!!!!!!Лучший Recall = 0.5031193372781423 

54/54 [==============================] - 3s 29ms/step - loss: 7.8876 - dense_10_loss: 1.4279 - dense_11_loss: 0.1588 - dense_12_loss: 4.7171 - dense_13_loss: 1.5838 - dense_10_accuracy: 0.3986 - dense_11_accuracy: 0.8400 - dense_12_accuracy: 0.1173 - dense_13_accuracy: 0.4291 - val_loss: 3.7923 - val_dense_10_loss: 0.2734 - val_dense_11_loss: 0.0000e+00 - val_dense_12_loss: 2.7620 - val_dense_13_loss: 0.7569 - val_dense_10_accuracy: 0.6222 - val_dense_11_accuracy: 1.0000 - val_dense_12_accuracy: 0.2889 - val_dense_13_accuracy: 0.4778
Epoch 2/20
53/54 [============================>.] - ETA: 0s - loss: 7.7755 - dense_10_loss:

ПРОВЕРИМ RECALL

In [ ]:
# чтение весов
model.load_weights('/content/drive/My Drive/Bases/DTP/model_22.h5')

In [ ]:
y_pred = model.predict(x_val)
y_p0 = np.around(y_pred[0]).astype(int)
y_p1 = np.around(y_pred[1]).astype(int)
y_p2 = np.around(y_pred[2]).astype(int)
y_p3 = np.around(y_pred[3]).astype(int)
y_t0 = np.around(y_val[:, 0]).astype(int)
y_t1 = np.around(y_val[:, 1]).astype(int)
y_t2 = np.around(y_val[:, 2]).astype(int)
y_t3 = np.around(y_val[:, 3]).astype(int)

score0 = recall_score(y_t0, y_p0, average='macro')
score1 = recall_score(y_t1, y_p1, average='macro')
score2 = recall_score(y_t2, y_p2, average='macro')
score3 = recall_score(y_t3, y_p3, average='macro')

print(score0, score1, score2, score3)
print((score0 + score1 + score2 + score3) / 4)

0.36063492063492064 1.0 0.1299382716049383 0.5236325519344387
0.5035514360435744
